In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm

from probly.losses import EvidenceLowerBound
from probly.representation import Bayesian

In [2]:
transforms = T.Compose([T.ToTensor()])
train = torchvision.datasets.FashionMNIST(root='~/datasets', train=True, download=True, transform=transforms)
test = torchvision.datasets.FashionMNIST(root='~/datasets', train=False, download=True, transform=transforms)
train_loader = DataLoader(train, batch_size=256, shuffle=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False)

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 10)
        self.act = nn.ReLU()
        self.max_pool = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.act(self.max_pool(self.conv1(x)))
        x = self.act(self.max_pool(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

net = LeNet()
model = Bayesian(net)

In [4]:
epochs = 20
optimizer = optim.Adam(model.parameters())
criterion = EvidenceLowerBound(1e-5)
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets, model.kl_divergence)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Running loss: {running_loss / len(train_loader)}, KL: {model.kl_divergence.item()}")

# compute accuracy on test set
correct = 0
total = 0
model.eval()
for inputs, targets in test_loader:
    outputs = model(inputs)
    correct += (outputs.argmax(1) == targets).sum()
    total += targets.size(0)
print(f"Accuracy: {correct / total}")

  5%|▌         | 1/20 [00:08<02:37,  8.27s/it]

Epoch 1, Running loss: 2.3561857304674514, KL: 92668.828125


 10%|█         | 2/20 [00:16<02:29,  8.28s/it]

Epoch 2, Running loss: 1.7392033242164775, KL: 92112.0


 15%|█▌        | 3/20 [00:23<02:13,  7.88s/it]

...

 95%|█████████▌| 19/20 [02:20<00:07,  7.17s/it]

Epoch 19, Running loss: 1.1155941230185489, KL: 78027.9609375


100%|██████████| 20/20 [02:28<00:00,  7.43s/it]

Epoch 20, Running loss: 1.0981770977060845, KL: 76987.140625


Accuracy: 0.8680999875068665
